# ДЗ 3. A/B-тестирование, ранговые критерии, бутстрэп

Дедлайн: 19.12.2023 в 23:59

Вот и наступило время решать заключительное ДЗ в рамках курса ПСМО! Что вам нужно будет сделать (вкратце):

- Всё дз в рамках одного набора данных о рекламной платформе
- Сначала необходимо выбрать правильное сплитование данных из 3 предложенных, на основании значения которой будет происходить A/B тест. Обосновать выбор, проверив необходимые предпосылки однородности А и В выборок с помощью ранговых критериев и изученного статистического аппарата (2 балла)
- * Может приключиться такая ситуация, что выбранное сплитование не совсем подходит для проведения A/B, поэтому не поленитесь выбрать ту, для которой интерпретация каждого из ранговых критериев соответствует вашим исходным предположениям
- Провести A/B тест и подвести итоги со всеми статистиками, использовать техники с семинара для повышения чувствительности/корректности А/В теста (3.5 балла)
- Реализовать бутстрап как альтернативный метод проверки гипотеы, поставленной в А/В (3 балла)
- Содержательные выводы по итогам работы, интерпретирующие результат с бизнесовой точки зрения, как минимум 3-4 предложения (1.5 балла)

Бонусы!
- +1 балл к дз, если до проведения основной части работы вы реализуете ЕМ-алгоритм для разделения смеси распределений (подсказка: смесей от 2 до 4)
- +1 балл к дз, если реализуете CUPED в рамках анализа результатов А/В

P.s. Кажется, если вы хорошо поняли крайние 3 темы курса, на решение ДЗ и интерпретацию результатов у вас уйдёт не больше 3-4 часов. Это дз больше про понимание и умение интерпретировать полученное, а не про фундаментальную математику

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats

Что есть в датасете для выполнения домашки:
* id - идентификатор пользователя
* split_1 - вариант сплитования 1
* split_2 - вариант сплитования 2
* split_3 - вариант сплитования 3
* is_teenager - меньше ли 25 лет пользователю
* num_segments - количество сегментов интересов [футбол, баскетбол, киберспорт, etc.] у пользователя
* views - просмотры рекламы в рамках проведённого эксперимента
* clicks - клики по рекламе в рамках проведённого эксперимента
* CTR - views / clicks

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/ZolotarevStat/psmo_22_23/2023/df_hw3.csv')
data

,id,split_1,split_2,split_3,is_teenager,num_segments,views,clicks,ctr
0,0,A,B,A,0,15,62,3,0.048387
1,1,A,B,B,0,4,5,0,0.000000
2,2,B,A,B,1,14,47,1,0.021277
3,3,B,A,B,1,2,3,0,0.000000
4,4,B,A,B,0,11,50,0,0.000000
...,...,...,...,...,...,...,...,...,...
19995,19995,B,A,A,1,4,5,0,0.000000
19996,19996,B,A,B,1,2,7,0,0.000000
19997,19997,A,B,A,0,6,22,5,0.227273
19998,19998,A,A,B,0,10,52,0,0.000000


## Бонус: EM (1 балл)

Реализуйте ЕМ-алгоритм для разделения смеси распределений (подсказка: смесей от 2 до 4) исходных данных. Реализация поможет лучше интерпретировать полученные результаты по ходу всего ДЗ


## Проверяем однородность разбиений (2 балла)

В этой части задания необходимо выбрать правильный метод разбиения выборки из предложенных вариантов [split_1, split_2, split_3], проверяя следующие гипотезы [при проверке используйте как минимум 2 ранговых критерия]:
* гипотеза о сдвиге
* гипотеза о масштабе
* гипотеза о независимости

Однородность разбиений необходимо проверять, используя только те данные, которые есть у нас ДО проведения А/В теста.

Разбалловка:
* За реализацию кода +0.5 балла
* За корректный и *обоснованный* выбор признаков, на основе которых проверяется однородность, +0.5 балла (при неверном выборе признаков 0 баллов за последующие части этого блока)
* За интерпретацию полученных результатов проверки гипотез +0.5 балла
* За выбор правильного метода разбиения по итогам блока +0.5 балла

NOTE: Обратите внимание на формат данных, имеющихся о пользователе до проведения эксперимента И внимательно подумайте над тем, каким образом можно проверить гипотезу о независимости

In [ ]:
splits = ['split_1', 'split_2', 'split_3']

p_values_dict = dict()
for split in splits:
  #YOUR CODE IS HERE#

## Подводим результаты А/В (3.5 балла)

Теперь, когда мы выбрали подходящее сплитование для нашего набора данных, мы можем посмотреть на результирующие метрики, которые получились по результатам А/В теста. В рамках данной части необходимо сделать следующее:
1. Сформулировать нулевую и альтернативную гипотезу для теста (0.5)
2. Сравнить значения получившихся значений целевых метрик, формально проверить гипотезу и интерпретировать результат (0.5)
3. Реализовать хотя бы два из пройденных на семинаре методов для уточнения корректности полученных результатов, если они здесь применимы (2 балла)
3. * Бакетизация
3. * Пост-стратификация
3. * Дельта-метод
4. За красивую и хорошо интерпретируемую визуализацию 0.5 балла

## bootstrap (3 балла)

Ещё раз подведите итоги проведённого А/В теста, используя [бутстрэп](https://habr.com/ru/companies/X5Tech/articles/679842/)

1. Оцените разницу в метриках, используя наивный бутстрэп и прокомментируйте, изменился ли результат А/В теста (1 балл)
2. Оцените разницу в метриках, используя t-бутстрэп и прокомментируйте, изменился ли результат А/В теста (1 балл)
3. Оцените разницу в метриках, используя BCA бутстрэп и прокомментируйте, изменился ли результат А/В теста (1 балл)

## Выводы (1.5 балла)

Содержательные выводы по итогам работы, интерпретирующие результат с бизнесовой точки зрения, как минимум 3-4 предложения

**NOTE:** если вы неверно выберете фичу сплитования или интерпретация полученных результатов А/В окажется некорректной, то за этот пункт будет поставлено 0 баллов вне зависимости от объёма выводов

*YOUR TEXT IS HERE*

## Бонус: CUPED (1 балл)

Опишите понятным проверяющему языком, каким образом работает CUPED и реализуйте его для проведённого А/В теста